In [5]:
import pandas as pd
import numpy as np
import nltk
import re
import string

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_pickle('prophecy_df.pkl')

In [4]:
df['century'].value_counts(normalize = True)

10.0    0.106157
9.0     0.106157
8.0     0.106157
6.0     0.106157
5.0     0.106157
4.0     0.106157
3.0     0.106157
2.0     0.106157
1.0     0.106157
7.0     0.044586
Name: century, dtype: float64

In [13]:
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

df['prophecy'] = df['prophecy'].map(alphanumeric).map(punc_lower)

In [14]:
df.head()

,century,prophecy,prophecy_num
0,1.0,sitting alone at night in secret study it is...,1.0
1,1.0,the wand in the hand is placed in the middle o...,2.0
2,1.0,when the litters are overturned by the whirlwi...,3.0
3,1.0,in the world there will be made a king who wi...,4.0
4,1.0,they will be driven away for a long drawn out ...,5.0


In [16]:
cv = CountVectorizer(stop_words = 'english')

X = cv.fit_transform(df['prophecy'])

pd.DataFrame(X.toarray(), columns = cv.get_feature_names()).head()

,abandon,abandoned,abased,abbe,abbey,abbot,abbots,ablaze,able,abolish,...,ye,year,years,yellow,yes,yore,young,younger,youth,étampes
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
cv = CountVectorizer(ngram_range = (1, 2), binary = True, stop_words = 'english')

X = cv.fit_transform(df['prophecy'])

pd.DataFrame(X.toarray(), columns = cv.get_feature_names()).head()

,abandon,abandon great,abandon people,abandon place,abandon realm,abandon secret,abandon siege,abandoned,abandoned arab,abandoned place,...,younger land,younger selin,younger son,younger sons,younger young,youth,youth assault,youth barbaric,étampes,étampes ruse
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
